In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13732,2024-12-30,Irã Superliga,09:30,Palayesh Naft Abadan,Tabiat,2.53,1.42,155.5,1.80,1.78,4.5,1.88,1.72,YPjqZVWo,0.395257,0.704225,0.555556,0.561798,0.099482,99.260,21.924944,0.220884,2.4,1.341641,0.559017,125.80,1.284,0.263685,0.205362,22.0,110.470,30.809950,0.278899,3.0,0.000000,0.000000,101.20,1.212,0.193313,0.159499,70.0,74,92,1.70,1.10,108.940,138.362,0.397412,0.007901,0.062854,-0.08,-0.016,-95.625000,0.680998,0.7,0.019002,1.51,0.302,1.390728,0.631536,0.7,0.068464
13733,2024-12-30,Hungria Nacional,14:00,Szedeak,Zalakeramia,2.67,1.37,164.5,1.81,1.77,4.5,1.98,1.64,dYWaNlUt,0.374532,0.729927,0.552486,0.564972,0.104459,353.840,277.066932,0.783029,0.6,1.341641,2.236068,125.06,4.464,3.399688,0.761579,-74.0,243.906,234.780286,0.962585,1.8,1.643168,0.912871,88.20,3.610,3.928556,1.088243,15.0,74,70,1.69,1.26,201.136,303.476,0.455069,0.015801,0.132827,-2.38,-0.476,-3.508403,0.629425,0.4,-0.229425,-1.85,-0.370,-1.000000,0.276447,0.1,-0.176447
13734,2024-12-30,Hungria Nacional,14:00,OSE Lions,NKA Pecs,1.75,1.84,162.5,1.81,1.77,-2.5,1.98,1.63,CAbs57EC,0.571429,0.543478,0.552486,0.564972,0.114907,290.674,308.501261,1.061331,0.6,1.341641,2.236068,171.52,3.522,3.569169,1.013393,-67.0,233.230,140.763745,0.603540,0.6,1.341641,2.236068,187.20,3.380,2.441383,0.722303,-51.0,67,80,2.56,2.34,183.340,233.180,0.035454,0.015801,0.137112,-1.46,-0.292,-2.568493,0.644940,0.6,-0.044940,0.35,0.070,12.000000,0.420103,0.4,-0.020103
13735,2024-12-30,Hungria Nacional,14:00,KTE-Duna,Szolnoki Olaj,2.18,1.56,145.5,1.84,1.74,1.5,2.02,1.61,xljTSA6J,0.458716,0.641026,0.543478,0.574713,0.099741,227.580,115.514505,0.507578,1.2,1.643168,1.369306,224.45,2.994,1.002213,0.334740,-11.0,225.388,270.241710,1.199007,1.8,1.643168,0.912871,708.10,2.968,3.764687,1.268426,-7.0,67,73,3.35,9.70,194.260,242.244,0.234442,0.039503,0.159732,0.97,0.194,6.082474,0.572750,0.6,0.027250,-1.08,-0.216,-2.592593,0.434048,0.5,0.065952
13736,2024-12-30,Finlândia Korisliiga,13:30,UU-Korihait,Kobrat,1.14,4.44,165.5,1.92,1.75,-10.5,1.98,1.70,Sbzce6Ob,0.877193,0.225225,0.520833,0.571429,0.102418,246.950,162.173657,0.656706,1.8,1.643168,0.912871,164.56,3.004,2.132259,0.709807,10.0,286.728,55.159175,0.192375,0.0,0.000000,NaN,291.48,3.984,1.257768,0.315705,-54.0,88,84,1.87,3.47,177.042,369.488,0.836363,0.065509,0.107603,4.00,0.800,0.175000,0.582447,0.7,0.117553,-5.00,-1.000,-3.440000,0.230215,0.0,-0.230215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798,2024-12-30,China Wcba Feminina,08:30,Guangdong F,Shanxi Xing Rui F,1.47,2.39,158.5,1.92,1.75,-5.5,1.98,1.70,0SnXur45,0.680272,0.418410,0.520833,0.571429,0.098682,308.198,418.016120,1.356323,2.4,1.341641,0.559017,186.02,3.872,5.139384,1.327320,29.0,217.406,254.460092,1.170437,2.4,1.341641,0.559017,115.92,3.098,4.140630,1.336550,30.0,71,84,2.62,1.38,183.892,488.704,0.337066,0.065509,0.107603,3.24,0.648,0.725309,0.647509,0.7,0.052491,-2.10,-0.420,-3.309524,0.454277,0.6,0.145723
13799,2024-12-30,China Cba,09:00,Xinjiang,Beijing,1.47,2.42,207.5,1.83,1.83,-5.5,1.93,1.71,pMeIIvYr,0.680272,0.413223,0.546448,0.546448,0.093495,146.174,30.432709,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13756,13:35,Israel Superliga,Gilboa Galil,Hapoel Holon,2.57,Back Home
13767,20:30,Eua Ncaa,Auburn,Monmouth (N.J.),1.83,Back Home
13779,16:45,Espanha Acb,Joventut,Gran Canaria,1.78,Back Home
